In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import zipfile
zip_ref = zipfile.ZipFile('/content/gdrive/My Drive/Datasets/BaiThi2.zip')
zip_ref.extractall()

In [178]:
import os
import numpy as np


In [179]:
data_spam = os.listdir("/content/TrainData/spam/")
data_not_spam = os.listdir("/content/TrainData/notspam/")


In [180]:
dt_spam = []
label_spam = []
for i in data_spam:
  path = os.path.join("/content/TrainData/spam/", i)
  with open(path, 'r') as f:
    dt_spam.append(f.readlines())
    label_spam.append(int(0))
len(dt_spam)

18

In [181]:
dt_not_spam = []
label_not_spam = []
for i in data_not_spam:
  path = os.path.join("/content/TrainData/notspam/", i)
  with open(path, 'r') as f:
    dt_not_spam.append(f.readlines())
    label_not_spam.append(int(1))
len(dt_not_spam)

193

In [182]:
import random
data_train = dt_spam[:12]+dt_not_spam[:150]
random.seed(0)
random.shuffle(data_train)

label_train = label_spam[:12]+label_not_spam[:150]
random.seed(0)
random.shuffle(label_train)


data_test = dt_spam[12:]+dt_not_spam[150:]
label_test = label_spam[12:]+label_not_spam[150:]
print(len(data_train))
print(len(data_test))

162
49


text processing

In [183]:
import string
import re
import sklearn
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS

In [184]:
from nltk.stem.porter import PorterStemmer

In [233]:
#cleaning the raw data
def remove_hyperlink(word):
    return  re.sub(r"http\S+", "", word)

def to_lower(word):
    result = word.lower()
    return result

def remove_number(word):
    result = re.sub(r'\d+', '', word)
    return result

def remove_punctuation(word):
    result = word.translate(str.maketrans(dict.fromkeys(string.punctuation)))
    return result

def remove_whitespace(word):
    result = word.strip()
    s = result.split()
    return (' '.join (str (x) for x in s))

def skimming(word):
  stemmer = PorterStemmer()
  singles = [stemmer.stem(x) for x in word.split()]
  return(' '.join(singles))

def replace_newline(word):
    return word.replace('\n','')


def clean_up_pipeline(sentence):
    cleaning_utils = [remove_hyperlink,
                      replace_newline,
                      to_lower,
                      remove_number,
                      remove_punctuation,remove_whitespace,
                      skimming]
    for o in cleaning_utils:
        sentence = o(sentence)
    return sentence

In [234]:
X_train = [clean_up_pipeline(str(x)) for x in data_train]
X_test = [clean_up_pipeline(str(x)) for x in data_test]

X_train[0]

'subject re languag acquisit analog pragmaticsn n ye inde care star sentenc sentenc star doe mean passerbi one even brentwood sever week ago wit cruel doubl murder testimoni broken english requir servic profession linguist interpret latter wa nt sure onc o j simpson star sentenc result is futur histori jule levin n'

2. Tokenize

In [235]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()

tokenizer.fit_on_texts(X_train)

x_train_features = np.array(tokenizer.texts_to_sequences(X_train))
x_test_features = np.array(tokenizer.texts_to_sequences(X_test))

len(x_train_features[0])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  import sys


52

In [ ]:
for i in range(5):
    print("Text               : ",X_train[i] )
    print("Numerical Sequence : ", x_train_features[i])

In [ ]:
tokenizer.index_word

In [238]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(label_train)
y_test = le.transform(label_test)
y_train

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1])

In [239]:
embed_size = 20 # how big is each word vector
max_feature = len(tokenizer.word_index)+1
max_len = max_length_sequence = max([len(i) for i in x_train_features])
print("max_feature", max_feature)
print("max_len", max_len)

max_feature 7016
max_len 2277


In [240]:
from keras_preprocessing.sequence import pad_sequences
x_train_features = pad_sequences(x_train_features,maxlen=max_len)
x_test_features = pad_sequences(x_test_features,maxlen=max_len)
x_train_features[0]

array([   0,    0,    0, ..., 2276, 3421,    2], dtype=int32)

4.Model

In [241]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional
from keras.models import Model
import tensorflow as tf

In [242]:
def create_model():
    
      lstm_model = tf.keras.Sequential()
      lstm_model.add(Embedding(max_feature, 32, input_length=max_len))
      lstm_model.add(LSTM(64))
      lstm_model.add(Dropout(0.5))
      lstm_model.add(Dense(20, activation="relu"))
      lstm_model.add(Dropout(0.4))
      lstm_model.add(Dense(1, activation = "sigmoid"))
      return lstm_model

lstm_model = create_model()
lstm_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
lstm_model.summary()

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_34 (Embedding)    (None, 2277, 32)          224512    
                                                                 
 lstm_34 (LSTM)              (None, 64)                24832     
                                                                 
 dropout_68 (Dropout)        (None, 64)                0         
                                                                 
 dense_68 (Dense)            (None, 20)                1300      
                                                                 
 dropout_69 (Dropout)        (None, 20)                0         
                                                                 
 dense_69 (Dense)            (None, 1)                 21        
                                                                 
Total params: 250,665
Trainable params: 250,665
Non-t

In [243]:
his = lstm_model.fit(x_train_features, y_train, batch_size=16, epochs=15, validation_data=(x_test_features, y_test))

Epoch 1/15
11/11 [==============================] - 4s 134ms/step - loss: 0.6797 - accuracy: 0.8148 - val_loss: 0.6554 - val_accuracy: 0.8776
Epoch 2/15
11/11 [==============================] - 1s 83ms/step - loss: 0.5900 - accuracy: 0.9198 - val_loss: 0.3743 - val_accuracy: 0.8776
Epoch 3/15
11/11 [==============================] - 1s 83ms/step - loss: 0.3567 - accuracy: 0.9321 - val_loss: 0.4711 - val_accuracy: 0.8776
Epoch 4/15
11/11 [==============================] - 1s 83ms/step - loss: 0.2995 - accuracy: 0.9259 - val_loss: 0.4501 - val_accuracy: 0.8776
Epoch 5/15
11/11 [==============================] - 1s 83ms/step - loss: 0.3303 - accuracy: 0.9259 - val_loss: 0.4056 - val_accuracy: 0.8776
Epoch 6/15
11/11 [==============================] - 1s 98ms/step - loss: 0.3113 - accuracy: 0.9259 - val_loss: 0.3897 - val_accuracy: 0.8776
Epoch 7/15
11/11 [==============================] - 1s 83ms/step - loss: 0.2770 - accuracy: 0.9259 - val_loss: 0.3954 - val_accuracy: 0.8776
Epoch 8/15
1

In [244]:
lstm_model.save('model_0.9796_2.h5')

In [245]:
y_predict  = [1 if x>=0.5 else 0 for x in lstm_model.predict(x_test_features)]

from sklearn.metrics import classification_report

print(classification_report(y_test,y_predict))

2/2 [==============================] - 1s 55ms/step
              precision    recall  f1-score   support

           0       1.00      0.83      0.91         6
           1       0.98      1.00      0.99        43

    accuracy                           0.98        49
   macro avg       0.99      0.92      0.95        49
weighted avg       0.98      0.98      0.98        49



label on test

In [262]:
from tensorflow import keras
model = keras.models.load_model('/content/model_0.9796_2.h5')

In [263]:
data_not_spam = os.listdir("/content/TestData_nolabel/")

test_nolabel = []
arr_nolabel=[]
for i in data_not_spam:
  path = os.path.join("/content/TestData_nolabel/", i)
  arr_nolabel.append(i)

len(test_nolabel)
arr_nolabel=sorted(arr_nolabel, key = lambda x: int(x[0]) if x[1]=='_' else int(x[0:2]))


In [264]:
for i in arr_nolabel:
  path = os.path.join("/content/TestData_nolabel/", i)
  with open(path, 'r') as f:
      test_nolabel.append(f.readlines())


In [265]:
arr_test = [clean_up_pipeline(str(x)) for x in test_nolabel]

In [266]:

from keras.preprocessing.text import Tokenizer

arr_test_feature = np.array(tokenizer.texts_to_sequences(arr_test))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


In [267]:
from keras_preprocessing.sequence import pad_sequences
arr_test_feature = pad_sequences(arr_test_feature,maxlen=max_len)


In [270]:
y_predict_label  = [1 if x>=0.5 else 0 for x in model.predict(arr_test_feature)]

3/3 [==============================] - 1s 57ms/step


In [ ]:
y_predict_label.count(0)

In [259]:
with open('/content/label_predict.txt', 'w+') as f:
     
    # write elements of list
    for items in y_predict_label:
        f.write('%s\n' %items)
     
    print("File written successfully")
 
 
# close the file
f.close()

File written successfully


In [260]:
arr_label=list()
with open('/content/label_predict.txt', 'r') as f:
  arr_label.append(f.readlines())
arr_label = np.reshape(arr_label,(78))
arr_la = []
for i in arr_label:
  arr_la.append(int(i[0:1]))
arr_la = ["notspam" if x==1 else "spam" for x in arr_la]
len(arr_la)

78

In [261]:
for i in range(78):
  with open('/content/file_nhan_0.9796.txt', 'w+') as f:
     
    # write elements of list
    for i in range(78):
        f.write('%s' % arr_nolabel[i])
        f.write(",")
        f.write('%s' % arr_la[i])
        f.write('\n')

     
print("File written successfully")

File written successfully
